In [3]:
%%capture
pip install --upgrade firebase-admin 

In [2]:
import requests
from bs4 import BeautifulSoup
import datetime
import time
import threading
import queue
import random

In [3]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

if not firebase_admin._apps:
    cred = credentials.Certificate("../jsonFile/service-account.json")
    firebase_admin.initialize_app(cred)
    
db = firestore.client()

In [4]:
def currentTime():
    now = datetime.datetime.now()
    currentTime = now.strftime("%Y-%m-%d %H:%M:%S")
    return currentTime

In [5]:
#personal detail
def personalData(soup):
    info = {}
    content = ''.join(soup.find('meta', property = 'og:image')['content']) #list to string

    d = soup.find('div', id = 'gsc_prf_i')

    #name
    info['name'] = d.find('div', id = 'gsc_prf_in').text

    #university
    try:
        info['university'] = d.find('a', class_ = 'gsc_prf_ila').text
    except:
        info['university'] = ' '

    #picture
    info['picture'] = soup.find('div', id = 'gsc_prf_pua').find('img')['src'] 
    
    label = []
    for p in soup.find_all('a', class_='gsc_prf_inta gs_ibl'):
        pre= p.text.replace(" ", "_")
        
        label.append(pre)

    info['label'] = label
    info['updateTime'] = currentTime()
    return info


In [6]:
def citeBy(soup):
    citeBy = {}
    citations = {}
    h_index = {}
    i10_index = {}
    def cited(status, value):

        if status / 2 < 1:        
            if status % 2 == 0:
                citations['All'] = value
            else:
                citations['Since2016'] = value
            citeBy['citations'] = citations

        if status / 2 < 2:
            if status % 2 == 0:
                h_index['All'] = value
            else:
                h_index['Since2016'] = value
            citeBy['h_index'] = h_index

        if status / 2 < 3:
            if status % 2 == 0:
                i10_index['All'] = value
            else:
                i10_index['Since2016'] = value
            citeBy['i10_index'] = i10_index
            
    count_d = 0
    for d in soup.find_all('td', class_='gsc_rsb_std'):

        cited(count_d, d.text)
        count_d = count_d + 1       
    
    return citeBy

In [7]:
def result(soup, ID):
    infos = {}
    infos['id'] = ID
    infos['personalData'] = personalData(soup)
    infos['cited'] = citeBy(soup)

    return infos

In [8]:
def personalPage(id):
    url = 'https://scholar.google.com.tw/citations?hl=zh-TW&user=' + id
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    return result(soup, id)

In [9]:
def test(id):
    url = 'https://scholar.google.com.tw/citations?hl=zh-TW&user=' + id
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    return result(soup, id)

In [10]:
p = test('JG-n5IcAAAAJ')
print(p)

{'id': 'JG-n5IcAAAAJ', 'personalData': {'name': 'Graham Dunn', 'university': 'University of Manchester', 'picture': '/citations/images/avatar_scholar_128.png', 'label': ['Medical_Statistics', 'Biostatistics', 'Measurement_Errors', 'Causal_Inference'], 'updateTime': '2021-10-19 09:41:13'}, 'cited': {'citations': {'All': '50982', 'Since2016': '19074'}, 'h_index': {'All': '117', 'Since2016': '74'}, 'i10_index': {'All': '360', 'Since2016': '271'}}}


In [11]:
def check_personalInfoFormat(data):
    #check id format
    rewriteData = data
    personalData = data['personalData']
    if (data['id'][7::] != 'AAAAJ' or data['id'] == ' ' or data['id'] == None):
        return False
    #check personal data format
    if(personalData['name'] == ' 'or personalData['name'] == None):
        return False        
    if(personalData['university'] == ' ' or personalData['university'] == '首頁' or personalData['university'] == None):
        rewriteData['personalData']['university'] = 'None'

    if(personalData['picture'] ==' ' or personalData['picture'] == None or personalData['picture'] == '/citations/images/avatar_scholar_128.png'):
        rewriteData['personalData']['picture'] = 'https://scholar.google.com.tw/citations/images/avatar_scholar_128.png'
    
    if(len(personalData['label']) == 0):
        rewriteData['personalData']['label'] = 'None'
    #check updateTime format 
    if(personalData['updateTime'] == '' or personalData['updateTime'] == None):
        rewriteData['personalData']['updateTime'] = currentTime()
    try:
        datetime.datetime.strptime(personalData['updateTime'], "%Y-%m-%d %H:%M:%S")
        
    except:
        rewriteData['personalData']['updateTime'] = currentTime()
    return rewriteData
       

        

In [12]:
e={'id': 'DZ-fHPAAAJ',
 'personalData': {'name': 'Bernhard Schölkopf',
  'university': 'None',
  'picture': 'https://scholar.googleusercontent.com/citations?view_op=view_photo&user=DZ-fHPgAAAAJ&citpid=2',
  'label': ['Machine Learning',
   'Causal Inference',
   'Artificial Intelligence',
   'Computational Photography',
   'Statistics'],
  'updateTime': '2021-10-09 13:52:05'},
 'cited': {'citations': {'All': '177951', 'Since2016': '80835'},
  'h_index': {'All': '163', 'Since2016': '115'},
  'i10_index': {'All': '577', 'Since2016': '484'}}}

In [69]:
check_personalInfoFormat(e)

False

In [55]:
def update_label(label):
    for i in label:
        ref = db.collection(u'Label-Domain').document(i)
        ref.set({u'updateTime': None})
   

In [56]:
update_label(p['personalData']['label'])

In [67]:
def get_LastUpdateLabel(limit):
    list = []
    query = db.collection(u'Label-Domain').order_by(u'updateTime').limit(limit)
    results = query.stream()
    for r in results:
        list.append(r.id)
    return list

['Biostatistics', 'Causal Inference', 'Measurement Errors']


In [8]:
import json
def jsontransfer(items):
    data = json.dumps(items)
    jsonStr = json.loads(data) 
    return jsonStr

In [9]:
def updatePersonal(userID):
    items = jsontransfer(personalPage(userID))
    print(items)
    ref = db.collection(u'cguscholar').document((items['id']))
    ref.collection(u'updateTime').document((items['personalData']['updateTime'])).set(items['cited'])
    ref.set(items['personalData'])

In [10]:
def IDCheck(ID):
    users_ref = db.collection(u'cguscholar').document(ID)
    doc = users_ref.get()
    if doc.exists:
        checkTemp = doc.to_dict()
        Timestamp = checkTemp['updateTime']
        return Timestamp
    else:
        return ('Not found')

In [11]:
def expiresCheck(last_update, expires):
    if last_update == 'Not found':
        compare = True
    else:
        expires_date = datetime.datetime.strptime(last_update, "%Y-%m-%d %H:%M:%S")
        compare_date = expires_date + datetime.timedelta(days = expires)
        current_date = datetime.datetime.now()

        compare = compare_date < current_date
    print(compare)
    return compare # compare result
    #過期或Not found為true

In [12]:
def ID_queue(label):
    # 建立佇列
    ID_queue = queue.Queue()

    # 將資料放入佇列
    label_ref = db.collection(u'Label-Domain').document(label)
    docs = label_ref.get()
    IDtemp = docs.to_dict()
    
    #取五個過期或為爬過的ID
    number = 5 
    ID_count = 0
    while (number != 0):        
        expire_time = IDCheck(IDtemp['userID'][ID_count])
        if(expiresCheck(expire_time, 1)):
            print(IDtemp['userID'][ID_count])
            ID_queue.put(IDtemp['userID'][ID_count])
            number = number - 1
        ID_count = ID_count + 1
        
    return ID_queue

In [13]:
def userDataList(label):
    
    # Worker 類別，負責處理資料
    class Worker(threading.Thread):
      def __init__(self, queue, num):
        threading.Thread.__init__(self)
        self.queue = queue
        self.num = num

      def run(self):
        while self.queue.qsize() > 0:
          # 取得新的資料
          userID = self.queue.get()

          # 處理資料
          #print("Worker %d: %s" % (self.num, userID))
          updatePersonal(userID)

          time.sleep(1)
    work_queue = ID_queue(label)
    # 建立兩個 Worker
    my_worker1 = Worker(work_queue, 1)
    my_worker2 = Worker(work_queue, 2)

    # 讓 Worker 開始處理資料
    my_worker1.start()
    my_worker2.start()

    # 等待所有 Worker 結束
    my_worker1.join()
    my_worker2.join()

    print("Done.")
    


In [ ]:
label = 'causal_inference'
userDataList(label)

In [14]:
def get_LabelForCGUScholar():
    query = db.collection(u'Label-Domain').where(u'updateTime', '==', None).limit(1)
    results = query.stream()
    for r in results:
        label = r.id
    return label

In [15]:
get_LabelForCGUScholar()

'5G'

In [19]:
def update_labeldomain(label):
    for i in label:
        ref = db.collection(u'Label-Domain').document(i)
        doc = ref.get()
        if doc.exists:
            continue
        else:
            ref.set({u'updateTime': None})
            print('exist')

In [20]:
p=['causal_inference']

In [21]:
update_labeldomain(p)

In [19]:

    query = db.collection(u'cguscholar').where(u'label', '==','causal_inference')
    results = query.stream()
    for r in results:
        label = r.id
    label

NameError: name 'label' is not defined

In [ ]:
replace("is", "was")

In [88]:
# jsonFile = open("jsonFile/personalData.json", "w")
# jsonFile.write(jsonStr)
# jsonFile.close()

In [3]:
t='salop '
if ' ' or '-' in t :
    print('v')

v


In [2]:
import time
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [9]:

urlt = 'https://scholar.google.com.tw/citations?hl=zh-TW&user=QHui70cAAAAJ'

def urlcontent(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36', 'Connention': 'close'
    }
    proxies={"https": "http://0.0.0.0:8888"}
    while 1:
        try:
            session = requests.Session()
            retry = Retry(connect=3, backoff_factor=0.5)
            adapter = HTTPAdapter(max_retries=retry)
            session.mount('http://', adapter)
            session.mount('https://', adapter)
            session.proxies.update(proxies)

            r = session.get(url, headers=headers)
            r=r.text
            break
        except Exception as e:
            print("request error.sleep 1 min and restart" + str(e))
            time.sleep(60)
            continue

    soup = BeautifulSoup(r, "html.parser")
    return soup
urlcontent(urlt)

request error.sleep 1 min and restartHTTPSConnectionPool(host='scholar.google.com.tw', port=443): Max retries exceeded with url: /citations?hl=zh-TW&user=QHui70cAAAAJ (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001D98814C970>: Failed to establish a new connection: [WinError 10049] 內容中所要求的位址不正確。')))


KeyboardInterrupt: 